## Импорт библиотек

In [62]:
import pandas as pd
import numpy as np
import math
from typing import Optional, Tuple, List
import statistics
from datetime import datetime, timedelta
from detecta import detect_seq
from itertools import groupby, islice
from plotly import graph_objs as go
from tqdm import tqdm
import bisect

## Чтение данных

### Телеметрия

In [68]:
data = pd.read_csv(r'data_validaion_calc\gas_tm_data.csv')
data['dt'] = pd.to_datetime(data['dt'])
data = data[data.param_id.isin([135, 142, 6001])]
print(f'Количество скважин: {len(data.well_id.unique())}')
data

Количество скважин: 16


,well_id,param_id,dt,val,ext_data,inserted
0,261,142,2022-11-20 00:00:07,44.4206,0,2023-11-19 19:01:34
1,261,142,2022-11-20 00:00:15,44.4763,0,2023-11-19 19:01:34
2,261,142,2022-11-20 00:00:24,44.4206,0,2023-11-19 19:02:05
3,261,142,2022-11-20 00:01:17,44.4763,0,2023-11-19 19:02:05
4,261,142,2022-11-20 00:01:26,44.4206,0,2023-11-19 19:03:33
...,...,...,...,...,...,...
3330848,963,6001,2022-12-15 23:59:16,73.3462,0,2023-12-15 19:01:02
3330849,963,6001,2022-12-15 23:59:25,73.4020,0,2023-12-15 19:01:02
3330850,963,6001,2022-12-15 23:59:34,73.3462,0,2023-12-15 19:01:02
3330851,963,6001,2022-12-15 23:59:42,73.4020,0,2023-12-15 19:01:02


### Уставки

In [69]:
data_controls = pd.read_csv(r'data_validaion_calc\tm_measure_control.csv')
data_controls

,well_id,param_id,min_value,max_value
0,693,135,72.0,108.0
1,693,142,54.0,126.0
2,693,6001,82.4,123.6
3,850,135,72.0,108.0
4,850,142,54.0,126.0
5,850,6001,78.4,117.6
6,86,135,60.0,90.0
7,86,142,30.0,34.0
8,86,6001,56.0,84.0
9,636,135,72.0,108.0


# Методы Валидации

### Удовлетворение значений технологическим уставкам (ТУ)

In [70]:
def add_min_max(validation_rows: list, controls: list) -> list:
    for row in validation_rows:
        param_controls = [x for x in controls if x["param_id"] == row["param_id"]]
        if len(param_controls) > 0:
            min_value = param_controls[0]["min_value"]
            max_value = param_controls[0]["max_value"]
            val = row["val"]
            if (min_value is not None and val < min_value) or (max_value is not None and val > max_value):
                row["min_max"] = 1
            else:
                row["min_max"] = 0
    return validation_rows

### Удовлетворение значений физическим границам (ФГ)

In [71]:
def phys_char_validate(validation_rows: list) -> list:
    """Проверка физических границ

    :param validation_rows: список словарей
    
    """
    char_limits = VALIDATION_PHYS_CHAR_DICT_DEFAULT
    for row in validation_rows:
        row["phys_bound"] = check(char_limits, row["param_id"], row["val"])
    return validation_rows


def check(char_limits: dict, param_id: int, value: float) -> int:
    """Проверка значения характеристики принадлежности интервалу её физических границ

    :param char_limits: физические границы
    :param param_id: идентификатор характеристики
    :param value: значение характеристики
    """
    return 1 if char_limits[param_id]["upper"] < value or char_limits[param_id]["low"] > value else 0

## Медианный фильтр

In [72]:
def nn_round(value: Optional[float], ndigits: int = 3) -> Optional[float]:
    if value is None or math.isnan(value) or math.isinf(value):
        return None
    else:
        return round(value, ndigits)


def apply_medfilt(
    measures: list,
    outlier_size: int,
    interval_koef: float,
    min_sigma: float,
) -> list:
    """
    Вызов алгоритма валидации - медианного фильтра для ПДФ скважин

    :param measures: список замеров
    :param outlier_size: кол-во выбросов после которого они войдут в обработку алгоритмом
    :param interval_koef: множитель интервала отклонения на расчетном окне
    :param min_sigma: минимальный множитель опимального отклонения
    
    :return: Список словарей
    """
    measures = sorted(measures, key=lambda d: d['dt'])
    if len(measures) == 0:
        return []
    kernel_size = 4 if len(measures) <= 5 else 5
    param_id = measures[0]["param_id"]
    ts = pd.Series([x["val"] for x in measures], index=[x["dt"] for x in measures])
    ts = ts.dropna()
    if len(ts) == 0 or len(ts) <= kernel_size:
        return []
    ts = ts.sort_index()
    # множитель интервала отклонения на расчетном окне
    ts_filter, sigma, outlier = newfilt_cumulative_time(
        ts,
        param_id,
        kernel_size=kernel_size,
        outlier_size=outlier_size,
        q=interval_koef,
        min_sigma=min_sigma,
    )

    for i, row in enumerate(measures):
        row.update(
            {
                "filter": nn_round(ts_filter[i]),
                "sigma": nn_round(sigma[i]),
                "outlier": outlier[i],
            }
        )

    return measures


def time_weight(df, last):
    timestep = last - df[0]
    timestep = timestep.total_seconds()
    weights = []
    for k in df:
        time_dif = last - k
        t = timestep - time_dif.total_seconds()
        weights.append(t / timestep)
    return np.array(weights)


def newfilt_cumulative_time(
    timeseries: pd.Series,
    param_id: int,
    kernel_size: int = 5,
    outlier_size: int = 7,
    q: int = 3,
    min_sigma: float = 0.05,
) -> Tuple[np.array, np.array, np.array]:
    """
    Модификация медианного фильтра.

    Аргументы:
    :param timeseries: -- временной ряд
    :param kernel_size: -- размер ядра
    :param outlier_size: -- число выбросов, после которых фильтр перестраивается
    :param q: -- множитель ширины интервала
    :param min_sigma: -- минимальное значение дисперсии по отношению к значениям. Иначе говоря,
        дисперсия не может быть меньше, чем min_sigma * x, где x -- значение ряда.

    :return: фильтрованный ряд, оценку дисперсии в каждый момент времени
    """

    indexs = np.arange(kernel_size)
    half_kernel_size = kernel_size // 2

    weights_med = np.exp(-np.abs(indexs - half_kernel_size) ** 2 / half_kernel_size)

    pressure_params = np.array([135, 6001, 142])

    dates = timeseries.index
    ts = np.array(timeseries)
    filter_vals = np.zeros_like(ts, dtype=float)
    sigma_vals = np.zeros_like(ts, dtype=float)
    outliers = np.zeros_like(ts, dtype=int)

    current_outliers = []
    dates_outliers = []
    ts_norm_last = []
    ts_filt_last = []

    dates_last = []

    # Инициализация
    for i in range(min(kernel_size, len(ts))):
        filter_vals[i] = np.median(ts[: i + 1])
        sigma_vals[i] = np.nan
        ts_norm_last = ts_norm_last[-kernel_size + 1 :] + [ts[i]]
        ts_filt_last = ts_filt_last[-kernel_size + 1 :] + [filter_vals[i]]
        dates_last = dates_last[-kernel_size + 1 :] + [dates[i]]

    # Итерации
    for i in range(kernel_size, len(ts)):
        # Исключение нулей для давлений
        if np.isin(param_id, pressure_params) and ts[i] <= 0:
            filter_vals[i] = filter_vals[i - 1]
            sigma_vals[i] = sigma_vals[i - 1]
            outliers[i] = 1

        else:
            # Вычисление сглаженных значений и дисперсии
            ts_norm_last_sorted = np.sort(ts_norm_last)

            timed_weights = time_weight(dates_last, dates[i])
            weights_last = np.log1p(timed_weights)
            weights_last = weights_last * weights_med.sum() / weights_last.sum()

            sum_weight = weights_last.sum() + weights_med.sum()
            weights_med = weights_med / sum_weight
            weights_last = weights_last / sum_weight

            filter_ts = np.sum(weights_last * ts_norm_last + weights_med * ts_norm_last_sorted)
            sigma_ts = (
                max(
                    np.sum(
                        weights_last * (np.array(ts_norm_last) - ts_filt_last) ** 2
                        + weights_med * (ts_norm_last_sorted - ts_filt_last) ** 2
                    ),
                    np.abs(ts_filt_last[-1]) * min_sigma,
                )
                ** 0.5
            )

            delta = q * sigma_ts

            # Если точка не аномальная
            if np.abs(filter_ts - ts[i]) < delta:
                current_outliers = []
                dates_outliers = []
                filter_vals[i] = filter_ts
                sigma_vals[i] = sigma_ts
                ts_norm_last = ts_norm_last[1:] + [ts[i]]
                ts_filt_last = ts_filt_last[1:] + [filter_vals[i]]
                dates_last = dates_last[1:] + [dates[i]]

            # Если точка аномальная
            else:
                # Стало много аномальных
                if len(current_outliers) >= outlier_size:
                    ts_norm_last = ts_norm_last + current_outliers
                    ts_norm_last = ts_norm_last[-kernel_size:]
                    current_outliers = []

                    dates_last = dates_last + dates_outliers
                    dates_last = dates_last[-kernel_size:]
                    dates_outliers = []

                    # Перестройка текущего прогноза
                    ts_norm_last_sorted = np.sort(ts_norm_last)

                    timed_weights = time_weight(dates_last, dates[i])
                    weights_last = np.log1p(timed_weights)
                    weights_last = weights_last * weights_med.sum() / weights_last.sum()

                    sum_weight = weights_last.sum() + weights_med.sum()
                    weights_med = weights_med / sum_weight
                    weights_last = weights_last / sum_weight

                    filter_ts = np.sum(weights_last * ts_norm_last + weights_med * ts_norm_last_sorted)

                    for j in range(len(ts_filt_last)):
                        ts_filt_last[j] = statistics.mean(ts_norm_last)

                    sigma_ts = (
                        max(
                            np.sum(
                                weights_last * (np.array(ts_norm_last) - ts_filt_last) ** 2
                                + weights_med * (ts_norm_last_sorted - ts_filt_last) ** 2
                            ),
                            np.abs(ts_filt_last[-1]) * min_sigma,
                        )
                        ** 0.5
                    )

                    filter_vals[i] = filter_ts
                    sigma_vals[i] = sigma_ts
                    ts_norm_last = ts_norm_last[1:] + [ts[i]]
                    ts_filt_last = ts_filt_last[1:] + [filter_vals[i]]
                    dates_last = dates_last[1:] + [dates[i]]

                # Аномальных еще не так много
                else:
                    current_outliers.append(ts[i])
                    dates_outliers.append(dates[i])
                    filter_vals[i] = filter_vals[i - 1]
                    sigma_vals[i] = sigma_vals[i - 1]
                    outliers[i] = 1

    if np.isin(param_id, pressure_params):
        if np.mean(ts) == 0:
            for i in range(kernel_size, len(ts)):
                if 0 == ts[i]:
                    outliers[i] = 0
    
    
    return filter_vals, np.dot(q, sigma_vals), outliers

## Попарная валидация

In [73]:
def pairwise_validation_calc(function, left_data, right_data):
    """Аггрегатор применения правил

    :param function: правило
    :param left_data: левый параметр в правиле
    :param right_data: правый параметр в правиле
    """
    right_dates = [x["dt"] for x in right_data]
    for item in left_data:
        if not item["pairwise_validation"]:
            date_index = bisect.bisect_left(right_dates, item["dt"])

            if date_index == 0:
                item["pairwise_validation"] = function(item, right_data[date_index])

            elif date_index == len(right_data):
                item["pairwise_validation"] = function(item, right_data[date_index - 1])

            elif right_dates[date_index] == item["dt"]:
                item["pairwise_validation"] = function(item, right_data[date_index])

            else:
                item["pairwise_validation"] = function(item, right_data[date_index])

            item["pairwise_validation"] = item["pairwise_validation"]


def pairwise_validate(validation_rows: list) -> list:
    """Попарная валидация

    :param validation_rows: список словарей
                        -'well_id'
                        -'param_id'
                        -'dt'
                        -'val'
                        -'filter'
                        -'filter_on'
                        -'sigma'
                        -'sigma_on'
                        -'sensor_bug'
                        -'min_max'
                        -'outlier'
                        -'outlier_on'
    """
    p_linear = [x for x in validation_rows if x["param_id"] == 142]
    p_buff = [x for x in validation_rows if x["param_id"] == 135]

    df_lin = pd.DataFrame(p_linear)
    df_buf = pd.DataFrame(p_buff)
    mean_diff = 0
    if not df_lin.empty and not df_buf.empty:
        df_merge = pd.merge_asof(
            df_lin[['dt', 'val']], df_buf[['dt', 'val']], on='dt', direction="nearest", tolerance=pd.Timedelta("30sec")
        )
        df_merge = df_merge.dropna()
        df_merge['delta'] = df_merge['val_x']-df_merge['val_y']
        mean_diff = df_merge['delta'].mean()
    if mean_diff < 1:
        for row in validation_rows:
            if row['param_id'] in (142, 135):
                row['pairwise_validation'] = 0
        return validation_rows

    def linear_buffer_pressure(p_linear_data: dict, p_buffer_data: dict) -> int:
        """Правило подсветки точек буферного и линейного давлений, где Рлин > Рбуф более чем на 1.5 атм

        :param p_linear_data: линейноe давлениe, атм
        :param p_buffer_data: буферное давление, атм
        """
        return (
            1
            if p_linear_data["val"] - p_buffer_data["val"] > 1.5
            and (p_buffer_data["phys_bound"] != 1 and p_linear_data["phys_bound"] != 1)
            else 0
        )

    def buffer_linear_pressure(p_buffer_data: dict, p_linear_data: dict) -> int:
        """Правило подсветки точек буферного и линейного давлений, где Рлин > Рбуф более чем на 1.5 атм

        :param p_buffer_data: буферное давление, атм
        :param p_linear_data: линейноe давлениe, атм
        """
        return (
            1
            if p_linear_data["val"] - p_buffer_data["val"] > 1.5
            and (p_linear_data["phys_bound"] != 1 and p_buffer_data["phys_bound"] != 1)
            else 0
        )

    if p_linear and p_buff:
        pairwise_validation_calc(linear_buffer_pressure, p_linear, p_buff)
        pairwise_validation_calc(buffer_linear_pressure, p_buff, p_linear)
    return validation_rows

## Зависания

In [74]:
def is_digit_span_int(df: pd.DataFrame) -> bool:
    """
    Определение разрядности чисел (являются ли числа в датафрейме числами без дробной части)

    :param df: датафрейм данных

    :return: True или False в зависимости от результата проверки
    """
    df["delta"] = df.val - df.val.apply(int)
    mean_delta = df["delta"].max()
    return True if mean_delta == 0 else False


def create_bug_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Создание колонки отвечающей за повторение значения предыдущим значением.
    Значения в колонке 0 или 1. 0 - предыдущее значение такое же, 1 - нет

    :param df: датафрейм данных

    :return: дополненный датафрейм данных
    """
    my_column_changes = df["val"].shift() != df["val"]
    df["bug"] = my_column_changes
    df["bug"] = df["bug"].astype(int)
    return df


def detect_sens_bug(
    measures: list,
    measures_time: list,
    min_time_bug: datetime = timedelta(hours=2),
):
    """
    детектирование зависших датчиков

    :param measures: замеры
    :param measures_time: времена замеров
    :param min_time_bug: минимальное время валидации

    :return: дополненные результаты валидации
    """
    time_bugs = []
    MIN_SENSOR_BUG_DISCR = timedelta(minutes=30)
    df = pd.DataFrame(measures, columns=["dt", "val"])
    digit_int_key = is_digit_span_int(df)
    if not digit_int_key:
        df = create_bug_df(df)
        bug_res_list = detect_seq(df.bug.to_numpy(), 0, index=True, min_seq=2)
        for bug_res in bug_res_list:
            time_vals = measures_time[bug_res[0] : bug_res[1] + 1]
            if time_vals[-1] - time_vals[0] >= min_time_bug:
                df_loc = df.iloc[bug_res[0] : bug_res[1] + 1]
                interval_mean = df_loc.val.mean()
                if interval_mean != 0:
                    mean_time_delta = df_loc.dt.diff().mean()
                    if mean_time_delta < MIN_SENSOR_BUG_DISCR and len(df_loc) >= 5:
                        time_bugs.extend(time_vals)
    for meas in measures:
        if meas["dt"] in time_bugs:
            meas["sensor_bug"] = 1
        else:
            meas["sensor_bug"] = 0

    return measures


def calc_sensor_bug(all_measures: list, param_ids: List[int]) -> List[dict]:
    results = []
    for param_id in param_ids:
        measures = []
        measures_val = []
        measures_time = []
        for x in all_measures:
            if x["param_id"] == param_id:
                measures.append(x)
                measures_val.append(x["val"])
                measures_time.append(x["dt"])
        if measures:
            check_val_in_row = any(
                sum(1 for _ in islice(g, 2)) > 1 for k, g in groupby(measures_val) if type(k) in [int, float]
            )
            if check_val_in_row:
                analyzed_rows = detect_sens_bug(measures, measures_time)
                for i, row in enumerate(measures):
                    row.update(
                        {
                            "sensor_bug": analyzed_rows[i]["sensor_bug"],
                        }
                    )
            else:
                for row in measures:
                    row.update(
                        {
                            "sensor_bug": 0,
                        }
                    )
            results.extend(measures)
    return results

### Создание начального формата данных

In [75]:
def validation_join_new(tm_data) -> List[dict]:
    result = []
    for row in tm_data:
        new_row = {
            "well_id": row["well_id"],
            "param_id": row["param_id"],
            "dt": row["dt"],
            "val": row["val"],
            "filter": None,
            "sigma": None,
            "outlier": 0,
            "sensor_bug": None,
            "min_max": None,
            "phys_bound": None,
            "pairwise_validation": None,
        }
        result.append(new_row)

    return result

### Подготовка данныx

In [22]:
def prepare_tm_day_interval_data(df_tm_data, well_id, dt_to):
    df_well_data = df_tm_data[(df_tm_data.well_id == well_id) & (df_tm_data.dt < dt_to)]
    param_ids = df_well_data.param_id.unique().tolist()
    df_well_day_data = df_well_data[df_well_data.dt >= dt_to - timedelta(days=1)]
    df_prev_data = df_well_data[df_well_data.dt < dt_to - timedelta(days=1)]
    append_dfs = []
    for param_id in param_ids:
        df_prev_data_param = df_prev_data[df_prev_data.param_id == param_id]
        df_prev_data_param = df_prev_data_param.sort_values('dt', ascending=False)
        df_prev_data_param = df_prev_data_param.iloc[0:10]
        append_dfs.append(df_prev_data_param)
    df_prev_append = pd.concat(append_dfs)
    result_df = pd.concat([df_well_day_data, df_prev_append])
    tm_data = result_df.to_dict('records')
    result_data = sorted(tm_data, key=lambda d: d['dt'])
    return result_data, param_ids

### Константы

In [76]:
MEDIAN_FILTER_SIGMAS = {
    135: 3,
    142: 3,
    6001: 3,
}

VALIDATION_PHYS_CHAR_DICT_DEFAULT = {
    135: {"low": 1, "upper": 200},  # атм
    142: {"low": 1, "upper": 200},  # атм
    6001: {"low": 1, "upper": 200},  # атм
}

param_name_dict = {
                  135: 'Р буф (ТМ)',
                  142: 'Р лин (ТМ)',
                  6001: 'Р затр (ТМ)',
                  7173: 'Р после клап (ТМ)',
                  7174: 'Т до клап (ТМ)',
                  7175: 'Т после клап (ТМ)',
                  7771: 'T газа в линии (ТМ)'
}

### Функции построения графиков

In [90]:
def create_well_graph(data, well_controls):
    fig = go.Figure()
    if not data:
        return fig
    params = list(set([x['param_id'] for x in data]))
    colors = ['rgba(225, 128, 0, 1)', 'rgba(0, 102, 0, 1)', 'rgba(0, 90, 204, 1)', 'rgba(153, 1, 153, 1)',  'rgba(102, 51, 0, 1)', 'rgba(204, 204, 0, 1)', 'rgba(204, 0, 102, 1)', 'rgba(0, 102, 102, 1)']
    colors_filter = ['rgba(225, 128, 20, 1)', 'rgba(0, 153, 10, 1)', 'rgba(5, 100, 255, 1)', 'rgba(255, 5, 255, 1)', 'rgba(102, 51, 15, 1)', 'rgba(204, 204, 15, 1)', 'rgba(204, 15, 102, 1)', 'rgba(15, 102, 102, 1)']
    colors_trust_interval = [
        'rgba(225, 128, 20, 0.25)', 'rgba(0, 153, 10, 0.25)', 'rgba(5, 100, 255, 0.25)', 'rgba(255, 5, 255, 0.25)','rgba(102, 51, 15, 0.25)', 'rgba(204, 204, 15, 0.25)', 'rgba(204, 15, 102, 0.25)', 'rgba(15, 102, 102, 0.25)'
    ]

    for i, param in enumerate(params[0:]):
        param_data = [x for x in data if x['param_id'] == param]
        param_data = sorted(param_data, key=lambda d: d['dt'])
        data_vals = [x['val'] for x in param_data]
        data_date = [x['dt'] for x in param_data]
        data_filter = np.array([x['filter'] for x in param_data])
        data_sigma = np.array([x['sigma'] for x in param_data])
        outlier_vals = [x['val'] for x in param_data if x['outlier'] == 1]
        min_max_vals = [x['val'] for x in param_data if x['min_max'] == 1]
        phys_bound_vals = [x['val'] for x in param_data if x['phys_bound'] == 1]
        sensor_bug_vals = [x['val'] for x in param_data if x['sensor_bug'] == 1]
        pairwise_vals = [x['val'] for x in param_data if x['pairwise_validation'] == 1]

        try:
            upper_boarder = data_filter + data_sigma
            lower_boarder = data_filter - data_sigma
            sum_date = data_date+data_date[::-1]
            concat_boarder = np.concatenate([lower_boarder,  upper_boarder[::-1]])
            fig.add_trace(go.Scattergl(
                x=sum_date,
                y=concat_boarder,
                fill='toself',
                fillcolor=colors_trust_interval[i],
                line=dict(width=0),
                mode="lines",
                name=f'{param_name_dict.get(param)} доверительный интервал'
            ))
        except Exception as e:
            print(f'Ошибка в графическом построении доверительного интервала {param}: {e}')

        fig.add_trace(go.Scattergl(
            x=data_date,
            y=data_vals,
            mode="lines+markers", name=f'{param_name_dict.get(param)}', line=dict(width=2, color=colors[i])),
            )
        fig.add_trace(go.Scattergl(
            x=data_date,
            y=data_filter,
            mode="lines", name=f'{param_name_dict.get(param)} фильтр', line=dict(width=1, color=colors_filter[i])),
            )
        if outlier_vals:
            outlier_date = [x['dt'] for x in param_data if x['outlier'] == 1]
            fig.add_trace(go.Scattergl(
                x=outlier_date,
                y=outlier_vals,
                mode="markers", name=f'{param_name_dict.get(param)} выбросы', line=dict(width=1.5, color='rgba(255, 0, 0, 1)')),
                )
        if min_max_vals:
            min_max_date = [x['dt'] for x in param_data if x['min_max'] == 1]
            fig.add_trace(go.Scattergl(
                x=min_max_date,
                y=min_max_vals,
                mode="markers", name=f'{param_name_dict.get(param)} выход за ТУ', line=dict(width=1.5, color='rgba(255, 0, 0, 1)')),
                )

        if phys_bound_vals:
            phys_bound_date = [x['dt'] for x in param_data if x['phys_bound'] == 1]
            fig.add_trace(go.Scattergl(
                x=phys_bound_date,
                y=phys_bound_vals,
                mode="markers", name=f'{param_name_dict.get(param)} выход за ФГ', line=dict(width=1.5, color='rgba(255, 0, 0, 1)')),
                )
        if sensor_bug_vals:
            sensor_bug_date = [x['dt'] for x in param_data if x['sensor_bug'] == 1]
            fig.add_trace(go.Scattergl(
                x=sensor_bug_date,
                y=sensor_bug_vals,
                mode="markers", name=f'{param_name_dict.get(param)} зависание', line=dict(width=1.5, color='rgba(255, 0, 0, 1)')),
                )
        if pairwise_vals:
            pairwise_date = [x['dt'] for x in param_data if x['pairwise_validation'] == 1]
            fig.add_trace(go.Scattergl(
                x=pairwise_date,
                y=pairwise_vals,
                mode="markers", name=f'{param_name_dict.get(param)} попарная валидация', line=dict(width=1.5, color='rgba(255, 0, 0, 1)')),
                )
        if well_controls:
            param_controls = [x for x in well_controls if x['param_id'] == param]
            if param_controls:
                param_controls = param_controls[0]
                min_val = param_controls['min_value']
                max_val = param_controls['max_value']
                min_date = param_data[0]['dt']
                max_date = param_data[-1]['dt']
                if min_val:
                    fig.add_trace(go.Scattergl(
                        x=[min_date, max_date],
                        y=[min_val, min_val],
                        mode="lines",
                        line=dict(width=1, color=colors_filter[i]),
                        name=f'{param_name_dict.get(param)} нижняя ТУ'
                    ))
                if max_val:
                    fig.add_trace(go.Scattergl(
                        x=[min_date, max_date],
                        y=[max_val, max_val],
                        mode="lines",
                        line=dict(width=1, color=colors_filter[i]),
                        name=f'{param_name_dict.get(param)} верхняя ТУ'
                    ))
    return fig


def plot_results(data, control_data, well_ids):
    figure_list = []
    for well_id in well_ids:
        well_controls = [x for x in control_data if x['well_id'] == well_id]
        well_data = [x for x in data if x['well_id'] == well_id]
        if well_data:
            fig = create_well_graph(well_data, well_controls)
            fig.show()

## Расчет дискретности

In [78]:
def discreteness_solver(data_list, param_ids, dt_to, well_id):
    df = pd.DataFrame(data_list)
    result = []
    for param in param_ids:
        df_param = df[(df.param_id == param)&(df.dt >= dt_to - timedelta(days=1))]
        if len(df_param) > 1:
            shift_dt = df_param.dt - df_param.dt.shift(1)
            shift_dt = shift_dt.dropna().apply(lambda x: x.total_seconds())
            median_discr = shift_dt.median()
            if median_discr:
                result.append({'well_id': well_id, 'param_id': param, 'dt_from': str(dt_to - timedelta(days=1)), 'dt_to': str(dt_to), 'val': median_discr})
    return result

### Функции расчета

In [79]:
def run_wells(df_tm_data, controls_data, well_ids, dt_to):
    result_calc = []
    result_discr = []
    for well_id in well_ids:
        tm_well_data, param_ids = prepare_tm_day_interval_data(df_tm_data, well_id, dt_to)
        well_discr = discreteness_solver(tm_well_data, param_ids, dt_to, well_id)
        data_well_controls = [x for x in controls_data if x['well_id'] == well_id]
        result = run_well(tm_well_data, data_well_controls, param_ids, dt_to)
        result_calc.extend(result)
        result_discr.extend(well_discr)
    return result_calc, result_discr


def run_well(tm_data, controls, param_ids, dt_to):
    validation_results = validation_join_new(tm_data)
    validation_results = add_min_max(validation_results, controls)
    validation_results = phys_char_validate(validation_results)
    validation_results = calc_sensor_bug(validation_results, param_ids)
    validation_results = pairwise_validate(validation_results)
    medfilt_param_ids = [x for x in MEDIAN_FILTER_SIGMAS if x in param_ids]
    no_medfilt_params = [x for x in validation_results if x['param_id'] not in medfilt_param_ids]
    medfilt_result = []
    for param_id in medfilt_param_ids:
        param_measures = [x for x in validation_results if x["param_id"] == param_id]
        outlier_size, interval_koef, min_sigma = 7, MEDIAN_FILTER_SIGMAS[param_id],  0.05
        medfilt_param_result = apply_medfilt(param_measures, outlier_size, interval_koef, min_sigma)
        medfilt_result.extend(medfilt_param_result)

    validation_results = medfilt_result + no_medfilt_params
    validation_results = [x for x in validation_results if x['dt'] >= dt_to - timedelta(days=1)]
    return validation_results

## Запуск расчета

In [80]:
wells_for_calc = data.well_id.unique()
control_data = data_controls.to_dict('records')
dt_from = datetime(2022, 12, 9, 0)
dt_to = datetime(2022, 12, 16, 0)
days = int((dt_to - dt_from).total_seconds()/86400)

all_results = []
all_discr_result = []
for i in tqdm(range(0, days)):
    dt_to_loc = dt_to - timedelta(days=i)
    result, result_discr = run_wells(data, control_data, wells_for_calc[:], dt_to_loc)
    all_results.extend(result)
    all_discr_result.extend(result_discr)


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:39<00:00, 14.20s/it]


## Графики результатов

In [92]:
all_results_graph = [x for x in all_results if x['dt'] > dt_to - timedelta(days=2)]

plot_results(all_results_graph, control_data, wells_for_calc[1:5])

### Сохранение результатов расчетов

In [82]:
df_results = pd.DataFrame(all_results)
df_results = df_results.sort_values(['well_id', 'param_id', 'dt'])
df_results = df_results.reset_index(drop=True)
df_results['dt'] = df_results['dt'].astype(str)
df_results['well_id'] = df_results['well_id'].astype(str)
df_results['param_id'] = df_results['param_id'].apply(lambda x: param_name_dict[x])
df_results['param_id'] = df_results['param_id'].astype(str)
df_results

,well_id,param_id,dt,val,filter,sigma,outlier,sensor_bug,min_max,phys_bound,pairwise_validation
0,86,Р буф (ТМ),2022-12-09 00:00:00,42.1100,42.229,4.358,0,0,1.0,0,1.0
1,86,Р буф (ТМ),2022-12-09 00:02:59,42.1100,42.202,4.359,0,0,1.0,0,1.0
2,86,Р буф (ТМ),2022-12-09 00:15:00,42.0900,42.181,4.358,0,0,1.0,0,1.0
3,86,Р буф (ТМ),2022-12-09 00:29:59,42.0700,42.132,4.357,0,0,1.0,0,1.0
4,86,Р буф (ТМ),2022-12-09 00:45:00,42.0400,42.099,4.354,0,0,1.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
598458,963,Р затр (ТМ),2022-12-15 23:59:16,73.3462,73.382,5.746,0,0,NaN,0,NaN
598459,963,Р затр (ТМ),2022-12-15 23:59:25,73.4020,73.364,5.746,0,0,NaN,0,NaN
598460,963,Р затр (ТМ),2022-12-15 23:59:34,73.3462,73.381,5.746,0,0,NaN,0,NaN
598461,963,Р затр (ТМ),2022-12-15 23:59:42,73.4020,73.365,5.746,0,0,NaN,0,NaN


In [83]:
df_stats = df_results[['well_id', 'param_id', 'outlier', 'sensor_bug', 'min_max', 'phys_bound', 'pairwise_validation']].groupby(
    by=['well_id', 'param_id']
).sum()
df_stats

outlier  sensor_bug  min_max  phys_bound  \
well_id param_id                                                
159     Р буф (ТМ)         0           0      0.0           0   
        Р затр (ТМ)        0           0      0.0           0   
        Р лин (ТМ)         0           0      0.0           0   
261     Р буф (ТМ)         0           0      0.0           0   
        Р затр (ТМ)        0           0      0.0           0   
        Р лин (ТМ)         0           0      0.0           0   
315     Р буф (ТМ)         0           0    100.0           0   
        Р затр (ТМ)        0           0      0.0           0   
        Р лин (ТМ)         0           0      0.0           0   
453     Р буф (ТМ)         0           0      0.0           0   
        Р затр (ТМ)        0           0      0.0           0   
        Р лин (ТМ)         0           0      0.0           0   
477     Р буф (ТМ)         0           0      0.0           0   
        Р затр (ТМ)        0           0      0.0           0   
        Р лин (ТМ)         0           0      0.0           0   
611     Р буф (ТМ)         0           0      0.0           0   
        Р затр (ТМ)        0           0      0.0           0   
        Р лин (ТМ)         0           0      0.0           0   
617     Р буф (ТМ)        46           0      0.0           1   
        Р затр (ТМ)        4          29      0.0           4   
        Р лин (ТМ)        35           0      0.0          15   
636     Р буф (ТМ)         0           0      0.0           0   
        Р затр (ТМ)        0           0      0.0           0   
        Р лин (ТМ)         0           0      0.0           0   
693     Р буф (ТМ)         3           0     26.0           0   
        Р затр (ТМ)        3           0     81.0           0   
        Р лин (ТМ)         0           0      0.0           0   
831     Р буф (ТМ)         0           0      0.0           0   
        Р затр (ТМ)        0           0      0.0           0   
        Р лин (ТМ)         0           0      0.0           0   
832     Р лин (ТМ)        49           0      0.0        1850   
850     Р буф (ТМ)        10           0     40.0           0   
        Р затр (ТМ)        7           0    325.0           0   
        Р лин (ТМ)         7           0      0.0           0   
86      Р буф (ТМ)        20           0    641.0           0   
        Р затр (ТМ)        1           0      0.0           0   
        Р лин (ТМ)        27           0    666.0           0   
885     Р лин (ТМ)         0           0      0.0           0   
954     Р буф (ТМ)        13           0      0.0           0   
        Р лин (ТМ)         0           0      0.0           0   
963     Р буф (ТМ)         0           0      0.0           0   
        Р затр (ТМ)        0           0      0.0           0   
        Р лин (ТМ)         0           0      0.0           0   

                     pairwise_validation  
well_id param_id                          
159     Р буф (ТМ)                   0.0  
        Р затр (ТМ)                  0.0  
        Р лин (ТМ)                   0.0  
261     Р буф (ТМ)                   0.0  
        Р затр (ТМ)                  0.0  
        Р лин (ТМ)                   0.0  
315     Р буф (ТМ)                   0.0  
        Р затр (ТМ)                  0.0  
        Р лин (ТМ)                   0.0  
453     Р буф (ТМ)                   0.0  
        Р затр (ТМ)                  0.0  
        Р лин (ТМ)                   0.0  
477     Р буф (ТМ)                   0.0  
        Р затр (ТМ)                  0.0  
        Р лин (ТМ)                   0.0  
611     Р буф (ТМ)                   0.0  
        Р затр (ТМ)                  0.0  
        Р лин (ТМ)                   0.0  
617     Р буф (ТМ)                   0.0  
        Р затр (ТМ)                  0.0  
        Р лин (ТМ)                   0.0  
636     Р буф (ТМ)                   0.0  
        Р затр (ТМ)                  0.0  
        Р лин (ТМ)  

In [84]:
df_discr = pd.DataFrame(all_discr_result)
df_discr['param_id'] = df_discr['param_id'].apply(lambda x: param_name_dict[x])
df_discr['dt_to'] = df_discr['dt_to'].astype(str)
df_discr = df_discr[['well_id', 'param_id', 'dt_from', 'dt_to', 'val']]
df_discr

,well_id,param_id,dt_from,dt_to,val
0,261,Р лин (ТМ),2022-12-15 00:00:00,2022-12-16 00:00:00,9.0
1,261,Р буф (ТМ),2022-12-15 00:00:00,2022-12-16 00:00:00,9.0
2,261,Р затр (ТМ),2022-12-15 00:00:00,2022-12-16 00:00:00,9.0
3,693,Р буф (ТМ),2022-12-15 00:00:00,2022-12-16 00:00:00,10132.5
4,693,Р лин (ТМ),2022-12-15 00:00:00,2022-12-16 00:00:00,14462.0
...,...,...,...,...,...
292,831,Р буф (ТМ),2022-12-09 00:00:00,2022-12-10 00:00:00,9.0
293,831,Р лин (ТМ),2022-12-09 00:00:00,2022-12-10 00:00:00,9.0
294,963,Р буф (ТМ),2022-12-09 00:00:00,2022-12-10 00:00:00,9.0
295,963,Р лин (ТМ),2022-12-09 00:00:00,2022-12-10 00:00:00,9.0
